In [31]:
# Importando as bibliotecas
import pandas as pd

In [32]:
# Lendo o arquivo .csv
autos = pd.read_csv('autos.csv', encoding='Latin-1')
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [35]:
# Renomeando colunas para snakecase e fazendo ajustes para melhor entendimento
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price',
                 'ab_test', 'vehicle_type', 'registration_year', 'gearbox',
                 'power_ps', 'model', 'odometer', 'registration_month',
                 'fuel_type', 'brand', 'unrepaired_damage', 'ad_created',
                 'num_photos', 'postal_code', 'last_seen']

In [36]:
# Removendo coluna com valores '0', e outras duas com muitos valores repetidos
autos = autos.drop(["num_photos", "seller", "offer_type"], axis=1)

In [37]:
# Ajustes para ver 'price' como um 'int'
autos["price"] = (autos["price"].str.replace("$", "")
                  .str.replace(",", "").astype(int))

In [38]:
# Ajustes na variável do odometro
autos["odometer"] = (autos["odometer"].str.replace("km", "")
                     .str.replace(",", "").astype(int))
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)

In [39]:
# Removendo registros com valores de ano de registro inválidos
autos = autos[autos["registration_year"].between(1900, 2016)]

In [40]:
# Analise de kilometragem e preço médio das "top 10" marcas anunciadas
common_brands = autos["brand"].value_counts(normalize=True).nlargest(10).index

In [43]:
# Analise precos
brand_mean_prices = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_price = brand_only["price"].mean()
    brand_mean_prices[brand] = int(mean_price)

mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)
mean_prices

mercedes_benz    30317
audi              9093
bmw               8334
ford              7263
volkswagen        6516
opel              5252
seat              4296
peugeot           3039
fiat              2711
renault           2395
dtype: int64

In [44]:
# Analise kilometragem
brand_mean_mileage = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mileage = brand_only["odometer_km"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)

mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_mileage

bmw              132434
mercedes_benz    130860
audi             129287
opel             129227
volkswagen       128730
renault          128183
peugeot          127136
ford             124046
seat             121563
fiat             116553
dtype: int64

In [45]:
# Dataframe final
brand_info = pd.DataFrame(mean_mileage, columns=['mean_mileage'])
brand_info["mean_price"] = mean_prices

brand_info

,mean_mileage,mean_price
bmw,132434,8334
mercedes_benz,130860,30317
audi,129287,9093
opel,129227,5252
volkswagen,128730,6516
renault,128183,2395
peugeot,127136,3039
ford,124046,7263
seat,121563,4296
fiat,116553,2711
